In [29]:
import tensorflow as tf
import numpy as np
import pretty_midi as midi
import os
import fluidsynth
import matplotlib.pyplot as plt
from tensorflow import keras 
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import time
import pandas as pd

In [30]:
TIME_STEPS = 35 
MIDI_PITCH_RANGE = 128

In [31]:
#midi_data is a list of PrettyMIDI objects
midi_data = []
directory = "data"

for filename in os.listdir(directory):
    midi_data.append(midi.PrettyMIDI(os.path.join(directory, filename)))
    

In [32]:
#Pull notes from midi file
velocities = []
pitches = []
starts = []
ends = []
durations = []
#Iterate through all midi objects, pull notes and zip together in a list 
for midi_obj in midi_data:
    piano = midi_obj.instruments[0]
    for note in piano.notes:
        velocities.append(note.velocity)
        pitches.append(note.pitch)
        starts.append(note.start)
        ends.append(note.end)
        durations.append(note.get_duration())

note_data_lists = list(zip(velocities, pitches, starts, ends, durations))

#Move zipped lists into dataframe and sort by start time so notes are chronological 
note_data = pd.DataFrame(note_data_lists, columns = ['Velocities', 'Pitches', 'Starts', 'Ends', 'Durations'])
note_data.sort_values(by = ["Starts"])


,Velocities,Pitches,Starts,Ends,Durations
0,80,69,0.980469,1.074219,0.093750
13697,45,69,0.983073,2.623698,1.640625
14201,49,68,0.985417,1.478125,0.492708
3,57,45,0.993490,1.407552,0.414062
16236,66,67,1.000000,1.104167,0.104167
...,...,...,...,...,...
22293,62,37,441.944792,443.887500,1.942708
22292,74,65,442.621875,443.868750,1.246875
22295,79,66,443.769792,449.291667,5.521875
22294,71,54,443.776042,449.029167,5.253125


In [59]:
train_data = note_data.to_numpy()
train_data = np.delete(train_data, 2, axis = 1)
train_data = np.delete(train_data, 3, axis = 1)
train_data = train_data[:-2, :]

In [65]:
pitches = 128
seq_length = 49 
num_features = 3

In [111]:
init_dataset = tf.data.Dataset.from_tensor_slices(train_data)

TensorSpec(shape=(3,), dtype=tf.float64, name=None)

In [118]:
seq_length = 26
windows = init_dataset.window(seq_length, shift = 1, stride = 1, drop_remainder = True)
map_func = lambda x : x.batch(seq_length, drop_remainder = True)
flattened_init_dataset = windows.flat_map(map_func)

In [121]:
flattened_init_dataset.element_spec

TensorSpec(shape=(26, 3), dtype=tf.float64, name=None)

ValueError: Attempt to convert a value (<FlatMapDataset element_spec=TensorSpec(shape=(26, 3), dtype=tf.float64, name=None)>) with an unsupported type (<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'>) to a Tensor.

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'element_spec'